In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/brown-corpus/brown.csv
/kaggle/input/brown-corpus/cats.csv
/kaggle/input/brown-corpus/brown-meta.json
/kaggle/input/brown-corpus/brown/brown/cb09
/kaggle/input/brown-corpus/brown/brown/cb10
/kaggle/input/brown-corpus/brown/brown/cj06
/kaggle/input/brown-corpus/brown/brown/cd17
/kaggle/input/brown-corpus/brown/brown/ca35
/kaggle/input/brown-corpus/brown/brown/cg20
/kaggle/input/brown-corpus/brown/brown/cf02
/kaggle/input/brown-corpus/brown/brown/cc17
/kaggle/input/brown-corpus/brown/brown/cl07
/kaggle/input/brown-corpus/brown/brown/cj74
/kaggle/input/brown-corpus/brown/brown/cg40
/kaggle/input/brown-corpus/brown/brown/cl15
/kaggle/input/brown-corpus/brown/brown/cj72
/kaggle/input/brown-corpus/brown/brown/cm03
/kaggle/input/brown-corpus/brown/brown/cc03
/kaggle/input/brown-corpus/brown/brown/cg72
/kaggle/input/brown-corpus/brown/brown/cj16
/kaggle/input/brown-corpus/brown/brown/ch02
/kaggle/input/brown-corpus/brown/brown/cg34
/kaggle/input/brown-corpus/brown/brown/cj42
/kag

In [2]:
import pandas as pd
brown_df = pd.read_csv("/kaggle/input/brown-corpus/brown.csv")

In [3]:
brown_df.head()

,filename,para_id,sent_id,raw_text,tokenized_text,tokenized_pos,label
0,cd05,0,0,"Furthermore/rb ,/, as/cs an/at encouragement/n...","Furthermore , as an encouragement to revisioni...","rb , cs at nn in nn nn , pps rb bez jj to vb c...",religion
1,cd05,0,1,The/at Unitarian/jj clergy/nns were/bed an/at ...,The Unitarian clergy were an exclusive club of...,at jj nns bed at jj nn in vbn nns -- cs at nn ...,religion
2,cd05,0,2,"Ezra/np Stiles/np Gannett/np ,/, an/at honorab...","Ezra Stiles Gannett , an honorable representat...","np np np , at jj nn in at nn , vbd ppl rb in a...",religion
3,cd05,0,3,"Even/rb so/rb ,/, Gannett/np judiciously/rb ar...","Even so , Gannett judiciously argued , the Ass...","rb rb , np rb vbd , at nn-tl md rb vb cs np ``...",religion
4,cd05,0,4,We/ppss today/nr are/ber not/* entitled/vbn to...,We today are not entitled to excoriate honest ...,ppss nr ber * vbn to vb jj nns wps vbd np to b...,religion


no NAs

In [4]:
brown_df.isna().sum()

filename          0
para_id           0
sent_id           0
raw_text          0
tokenized_text    0
tokenized_pos     0
label             0
dtype: int64

In [5]:
brown_df["tokenized_text"][0]

'Furthermore , as an encouragement to revisionist thinking , it manifestly is fair to admit that any fraternity has a constitutional right to refuse to accept persons it dislikes .'

In [6]:
from sklearn.model_selection import train_test_split

In [7]:
train_df, test_df = train_test_split(brown_df, test_size=0.2)

In [8]:
import pandas as pd
import torch
from torch.utils.data import Dataset
import ast
from collections import Counter 


class CBOW_Dataset_vOLD(Dataset):
    def __init__(self, df, window_size=2):
        # Load CSV

        # If tokenized_text is stored as a string like "['The','cat']", convert to list
        '''if isinstance(brown_df['tokenized_text'].iloc[0], str):t
            brown_df['tokenized_text'] = brown_df['tokenized_text'].apply(ast.literal_eval)'''

        self.data = df
        self.window_size = window_size


        all_tokens = []
        # Build vocabulary
        for row in data["tokenized_text"]:
            # Lowercase, remove punctuation, then split
            clean_text = re.sub(r'[^\w\s]', '', row.lower())
            split_text_list = clean_text.split()
            all_tokens.extend(split_text_list)

        counts = Counter(all_tokens)

        min_frequency = 5
        filtered_counts = Counter({item: count for item, count in counts.items() if count < min_frequency})

        self.vocab = [w for w, _ in filtered_counts.most_common()] #sort by most frequently used words 
        #self.vocab = sorted(set(all_tokens)) #sorting everything here helps with training. It will sort the words alphabetically. 
        self.word2idx = {w: i for i, w in enumerate(self.vocab)}
        self.idx2word = {i: w for w, i in self.word2idx.items()}
        print("length", len(self.word2idx))
        # Build (context, target) pairs once during initialization
        self.pairs = []
        
        for text in self.data['tokenized_text']: #for each row in tokenized_text 
            tokens = re.sub(r"[^\w\s]", "", text.lower()).split()  #create tokens - get rid of punctuation 
            for i in range(self.window_size, len(tokens) - self.window_size):
                context = tokens[i - self.window_size:i] + tokens[i + 1:i + self.window_size + 1]  #gets everything to the left and right of target token 
                target = tokens[i]   #gets target token 

                context_ids = [self.word2idx[w] for w in context if w in self.word2idx]
                if target in self.word2idx: 
                    target_id = self.word2idx[target]
                    
                context_tensor = torch.tensor(context_ids)
                target_tensor = torch.tensor(target_id)
                self.pairs.append((context_tensor, target_tensor))

    def __len__(self):
        return len(self.pairs)

    def get_word2idx(self): 
        return self.word2idx

    def get_idx2word(self): 
        return self.idx2word

    def __getitem__(self, idx):
        context_ids, target_id =  self.pairs[idx]
        return context_ids, target_id
        

In [9]:
import torch
from torch.utils.data import Dataset
import re
from collections import Counter

class CBOW_Dataset(Dataset):
    def __init__(self, df, window_size=2, min_frequency=5):
        self.data = df
        self.window_size = window_size

        # Step 1: tokenize text, remove punctuation, lowercase
        all_tokens = []
        for row in self.data["tokenized_text"]:
            clean_text = re.sub(r"[^\w\s]", "", row.lower())
            tokens = clean_text.split()
            all_tokens.extend(tokens)

        # Step 2: count word frequencies
        counts = Counter(all_tokens)

        # Step 3: keep only frequent words (>= min_frequency)
        vocab_words = [w for w, c in counts.items() if c >= min_frequency]

        # Step 4: sort vocabulary by descending frequency
        vocab_words = sorted(vocab_words, key=lambda w: counts[w], reverse=True)

        # Step 5: add <UNK> token
        vocab_words.append("<UNK>")

        # Step 6: build mappings
        self.vocab = vocab_words
        self.word2idx = {w: i for i, w in enumerate(self.vocab)}
        self.idx2word = {i: w for w, i in self.word2idx.items()}
        print("Vocab size:", len(self.vocab))

        # Step 7: build (context, target) pairs
        self.pairs = []
        for text in self.data['tokenized_text']:
            tokens = re.sub(r"[^\w\s]", "", text.lower()).split()
            for i in range(self.window_size, len(tokens) - self.window_size):
                context = tokens[i - self.window_size:i] + tokens[i + 1:i + self.window_size + 1]
                target = tokens[i]

                # convert context and target to indices, using <UNK> if missing
                context_ids = [self.word2idx[w] if w in self.word2idx else self.word2idx["<UNK>"] for w in context]
                target_id = self.word2idx[target] if target in self.word2idx else self.word2idx["<UNK>"]

                self.pairs.append((torch.tensor(context_ids, dtype=torch.long),
                                   torch.tensor(target_id, dtype=torch.long)))

    def __len__(self):
        return len(self.pairs)

    def __getitem__(self, idx):
        return self.pairs[idx]

    def get_word2idx(self):
        return self.word2idx

    def get_idx2word(self):
        return self.idx2word


In [10]:
from torch.nn import Module
import torch.nn as nn
from torch.nn import functional as F 
class CBOW_MODEL(Module): 
    def __init__(self, vocab_size, embedding_size, context_size):  #context size is how many words around the the center word we're predicting
        super().__init__()
        self.embedding_layer = nn.Embedding(vocab_size, embedding_size) #create embedding layer 
        self.fc1 = nn.Linear(embedding_size* context_size, 128) #check why we do embedding size * context size 
        self.fc2 = nn.Linear(128, vocab_size) #project to a word within the vocab size
    
    def forward(self, input):  
        input = self.embedding_layer(input) #after doing this, our input will have shape: (batch, context_size, d_model), we need to flatten it before passing into fc1
        input = input.view(input.size(0), -1) #flattens the embedding and context size. New tensor is (batch_size, embedding_size * context size). Note that this linear layer is taking a
        #two dimensional output, but it's okay. The linear transformation is perforemd on the last dimension of the tensor (in this case, embedding_size * context_size)
        input = self.fc1(input)
        input = F.relu(input)
        logits = self.fc2(input)

        logits = F.log_softmax(logits, dim=1)

        #input = F.softmax(input, dim=-1) already incorporated in loss 
        return logits
    


In [11]:
train_dataset = CBOW_Dataset(train_df)
test_dataset = CBOW_Dataset(test_df)


Vocab size: 12267
Vocab size: 4647


In [12]:
print(len(train_dataset))

633830


In [13]:
num_epochs = 15
batch_size = 32 

In [14]:
from torch.utils.data import DataLoader

In [15]:
train_loader = DataLoader(train_dataset, batch_size, num_workers=2, shuffle=False)

In [16]:
test_loader = DataLoader(test_dataset, 
                         batch_size = 32, 
                         num_workers = 2, 
                         shuffle=True
)


In [17]:
from torch import optim
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = CBOW_MODEL(vocab_size=12324, embedding_size=250, context_size=4).to(device)
criterion = nn.NLLLoss()
optimizer = optim.SGD(model.parameters(), lr = 0.001)

In [18]:
n_total_steps = len(train_loader) 
for epoch in range(num_epochs):
    for i, (context,target) in enumerate(train_loader): 
        
        context = context.to(device)
        target = target.to(device)
        outputs = model(context) 
        loss_value = criterion(outputs, target) 
        optimizer.zero_grad()
        loss_value.backward()
        optimizer.step()

        if(i+1) % 2000 ==0:
            print (f'Epoch [{epoch+1}/{num_epochs}], Step [{i+1}/{n_total_steps}], Loss: {loss_value.item():.4f}')


Epoch [1/15], Step [2000/19808], Loss: 9.1034
Epoch [1/15], Step [4000/19808], Loss: 8.4006
Epoch [1/15], Step [6000/19808], Loss: 8.7125
Epoch [1/15], Step [8000/19808], Loss: 6.9604
Epoch [1/15], Step [10000/19808], Loss: 7.6161
Epoch [1/15], Step [12000/19808], Loss: 8.1551
Epoch [1/15], Step [14000/19808], Loss: 7.9450
Epoch [1/15], Step [16000/19808], Loss: 6.8758
Epoch [1/15], Step [18000/19808], Loss: 6.6266
Epoch [2/15], Step [2000/19808], Loss: 6.9502
Epoch [2/15], Step [4000/19808], Loss: 7.3922
Epoch [2/15], Step [6000/19808], Loss: 7.8470
Epoch [2/15], Step [8000/19808], Loss: 5.9862
Epoch [2/15], Step [10000/19808], Loss: 6.8265
Epoch [2/15], Step [12000/19808], Loss: 7.2855
Epoch [2/15], Step [14000/19808], Loss: 7.4142
Epoch [2/15], Step [16000/19808], Loss: 6.4824
Epoch [2/15], Step [18000/19808], Loss: 5.8965
Epoch [3/15], Step [2000/19808], Loss: 6.7267
Epoch [3/15], Step [4000/19808], Loss: 7.0284
Epoch [3/15], Step [6000/19808], Loss: 7.4719
Epoch [3/15], Step [8000

In [19]:
word2idx = test_dataset.get_word2idx()
idx2word = test_dataset.get_idx2word()

In [20]:
model.eval()  # set model to evaluation mode
word2idx = test_dataset.get_word2idx()
idx2word = test_dataset.get_idx2word()


with torch.no_grad():
    n_correct = 0
    n_samples = 0
    vocab_size = len(word2idx)  # size of your vocabulary
    n_class_correct = [0 for _ in range(vocab_size)]
    n_class_samples = [0 for _ in range(vocab_size)]

    for context_idxs, target_idxs in test_loader:
        context_idxs = context_idxs.to(device)  # shape: (batch_size, context_size)
        target_idxs = target_idxs.to(device)    # shape: (batch_size,)

        outputs = model(context_idxs)  # shape: (batch_size, vocab_size)
        _, predicted = torch.max(outputs, dim=1)  # predicted word indices

        n_samples += target_idxs.size(0)
        n_correct += (predicted == target_idxs).sum().item()

        # per-word accuracy
        for i in range(len(target_idxs)):
            label = target_idxs[i].item()
            pred = predicted[i].item()
            if label == pred:
                n_class_correct[label] += 1
            n_class_samples[label] += 1

    overall_acc = 100.0 * n_correct / n_samples
    print(f'Overall accuracy: {overall_acc:.2f}%')

    # per-word accuracy (optional, can be slow for large vocab)
    for i in range(vocab_size):
        if n_class_samples[i] > 0:
            word_acc = 100.0 * n_class_correct[i] / n_class_samples[i]
            print(f'Accuracy of "{idx2word[i]}": {word_acc:.2f}%')


Overall accuracy: 8.64%
Accuracy of "the": 68.39%
Accuracy of "of": 45.99%
Accuracy of "and": 11.56%
Accuracy of "to": 22.48%
Accuracy of "a": 7.19%
Accuracy of "in": 16.27%
Accuracy of "that": 7.90%
Accuracy of "is": 6.86%
Accuracy of "for": 0.66%
Accuracy of "was": 0.63%
Accuracy of "he": 0.09%
Accuracy of "it": 8.69%
Accuracy of "as": 1.35%
Accuracy of "with": 4.78%
Accuracy of "his": 0.39%
Accuracy of "on": 0.42%
Accuracy of "be": 5.31%
Accuracy of "at": 0.78%
Accuracy of "i": 0.35%
Accuracy of "had": 0.00%
Accuracy of "this": 0.17%
Accuracy of "by": 0.00%
Accuracy of "not": 2.86%
Accuracy of "are": 0.00%
Accuracy of "but": 0.00%
Accuracy of "from": 0.13%
Accuracy of "or": 0.00%
Accuracy of "have": 10.26%
Accuracy of "an": 0.00%
Accuracy of "they": 0.20%
Accuracy of "which": 2.25%
Accuracy of "one": 0.00%
Accuracy of "you": 0.00%
Accuracy of "were": 0.19%
Accuracy of "all": 0.00%
Accuracy of "would": 0.00%
Accuracy of "her": 0.00%
Accuracy of "she": 0.00%
Accuracy of "him": 0.00%
A

In [24]:
torch.save(model.state_dict(),"cbow_model.pt")

In [25]:
embed_layer = model.embedding_layer.weight

In [29]:
import pickle

my_dictionary= idx2word

# Save the dictionary to a file
with open("idx2word.pkl", "wb") as file:
    pickle.dump(my_dictionary, file)

# Load the dictionary back from the file
with open("idx2word.pkl", "rb") as file:
    loaded_dictionary = pickle.load(file)

print(loaded_dictionary)


{0: 'the', 1: 'of', 2: 'and', 3: 'to', 4: 'a', 5: 'in', 6: 'that', 7: 'is', 8: 'for', 9: 'was', 10: 'he', 11: 'it', 12: 'as', 13: 'with', 14: 'his', 15: 'on', 16: 'be', 17: 'at', 18: 'i', 19: 'had', 20: 'this', 21: 'by', 22: 'not', 23: 'are', 24: 'but', 25: 'from', 26: 'or', 27: 'have', 28: 'an', 29: 'they', 30: 'which', 31: 'one', 32: 'you', 33: 'were', 34: 'all', 35: 'would', 36: 'her', 37: 'she', 38: 'him', 39: 'we', 40: 'their', 41: 'there', 42: 'been', 43: 'when', 44: 'has', 45: 'who', 46: 'if', 47: 'no', 48: 'out', 49: 'its', 50: 'more', 51: 'will', 52: 'so', 53: 'said', 54: 'what', 55: 'into', 56: 'about', 57: 'them', 58: 'than', 59: 'only', 60: 'time', 61: 'other', 62: 'up', 63: 'can', 64: 'some', 65: 'new', 66: 'these', 67: 'may', 68: 'any', 69: 'could', 70: 'my', 71: 'two', 72: 'such', 73: 'then', 74: 'do', 75: 'like', 76: 'first', 77: 'over', 78: 'me', 79: 'our', 80: 'now', 81: 'man', 82: 'even', 83: 'did', 84: 'must', 85: 'before', 86: 'made', 87: 'most', 88: 'back', 89: 'y

In [26]:
import torch
import torch.nn.functional as F

def closest_words(model, word, word_to_idx, idx_to_word, top_k=10):
    # 1. look up index of the word
    if word not in word_to_idx:
        return f"'{word}' not in vocabulary"

    word_idx = word_to_idx[word]

    # 2. get embedding for the query word (1 x d)
    word_vec = model.embedding_layer.weight[word_idx]  # (d,)
    word_vec = word_vec.unsqueeze(0)                   # (1, d)

    # 3. get all embeddings (vocab_size x d)
    all_embeds = model.embedding_layer.weight          # (V, d)

    # 4. compute cosine similarity with every word
    sims = F.cosine_similarity(word_vec, all_embeds)   # (V,)

    # 5. sort by similarity (descending)
    topk = torch.topk(sims, top_k + 1)                 # +1 to skip the word itself

    # 6. convert indices back to words
    results = []
    for idx in topk.indices:
        w = idx_to_word[int(idx)]
        if w != word:   # skip the query word
            results.append(w)

        if len(results) == top_k:
            break

    return results
    


In [ ]:
word2vec = model.get_word2vec